# Previsão de Probabilidade de Preços por Desvio Padrão

## 1. Importação das Bibliotecas e Configurações

In [2]:
%config IPCompleter.greedy=True
import yfinance as yf
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 2. Definição de Parâmetros

In [3]:
hoje = datetime.date.today()
inicio = hoje - datetime.timedelta(days = 365) # pegar 1 ano de dados
hoje = str(hoje)
inicio = str(inicio)

dias_venc = 15 #modificar colocando quantos dias faltam para o vencimento da opção ou quantos dias gostaria para o range da previsão de probabilidade


In [4]:
ativos = ["PETR4", "VALE3", "BOVA11", "MGLU3", "CIEL3", "CSNA3", "COGN3", "IRBR3", "VIIA3", "B3SA3", "BBAS3", "WEGE3", 
          "USIM5", "BRFS3", "BBDC4", "ITSA4", "ABEV3", "JBSS3", "PRIO3"] #pode incluir mais ativos
ativos = [i + '.SA' for i in ativos]


## 3. Baixando dados

In [5]:
df = yf.download(ativos, start = inicio, end = hoje)['Adj Close']

[*********************100%***********************]  19 of 19 completed


## 4. Tratamento os dados

In [6]:
retornos = df.pct_change()
retornos = retornos.dropna()
data_ref = datetime.date.today()-datetime.timedelta(days = 1) #usa o preço da data de ontem como referência 
data_ref = str(data_ref)
preco_ref = df.loc[data_ref]


## 5. Calculo do EWMA

In [7]:
length = len(retornos)
Lambda = 0.94
ser = np.arange(length-1,-1,-1)
exp = np.power(Lambda,ser)*(1-Lambda)
prod = pd.DataFrame()

for i in retornos:
  prod[i] = exp*retornos.loc[:,i]**2
  EWMA = np.sqrt(prod.sum())

## 6. Previsão para periodo

In [8]:
desvio = EWMA
media = retornos.mean()
medret = (((1+media)**(dias_venc))-1)
desvret = (desvio*np.sqrt(dias_venc))

## 7. Calculo dos preços de cada Desvio

In [9]:
desvio_05_p = round(preco_ref*(1+medret+0.5*desvret),2)
desvio_10_p = round(preco_ref*(1+medret+desvret),2)
desvio_15_p = round(preco_ref*(1+medret+1.5*desvret),2)
desvio_20_p = round(preco_ref*(1+medret+2*desvret),2)

desvio_05_n = round(preco_ref*(1+medret-0.5*desvret),2)
desvio_10_n = round(preco_ref*(1+medret-desvret),2)
desvio_15_n = round(preco_ref*(1+medret-1.5*desvret),2)
desvio_20_n = round(preco_ref*(1+medret-2*desvret),2)

## 8. Visualizando os valores dos desvios dos preços

In [10]:
desvios = pd.DataFrame([desvio_20_p, desvio_15_p, desvio_10_p, desvio_05_p, desvio_05_n, desvio_10_n, desvio_15_n, desvio_20_n])
desvios = desvios.rename({0: "2,0 desvios alta", 1: "1,5 desvios alta", 2: "1,0 desvios alta", 3: "0,5 desvios alta", 
                         7: "2,0 desvios baixa", 6: "1,5 desvios baixa", 5: "1,0 desvios baixa", 4: "0,5 desvios baixa"})
desvios

,ABEV3.SA,B3SA3.SA,BBAS3.SA,BBDC4.SA,BOVA11.SA,BRFS3.SA,CIEL3.SA,COGN3.SA,CSNA3.SA,IRBR3.SA,ITSA4.SA,JBSS3.SA,MGLU3.SA,PETR4.SA,PRIO3.SA,USIM5.SA,VALE3.SA,VIIA3.SA,WEGE3.SA
"2,0 desvios alta",18.00,15.69,42.10,18.16,120.37,10.34,5.74,2.64,19.02,0.86,9.77,25.86,4.31,32.34,44.54,9.43,105.95,2.73,45.07
"1,5 desvios alta",17.52,14.83,40.47,17.28,116.76,9.68,5.46,2.51,17.84,0.80,9.48,24.63,3.98,30.85,42.76,8.91,101.53,2.53,43.32
"1,0 desvios alta",17.04,13.98,38.84,16.40,113.15,9.03,5.18,2.38,16.67,0.73,9.19,23.41,3.65,29.36,40.97,8.40,97.11,2.34,41.58
"0,5 desvios alta",16.56,13.13,37.21,15.52,109.55,8.38,4.90,2.25,15.50,0.66,8.91,22.18,3.32,27.88,39.18,7.88,92.69,2.14,39.84
"0,5 desvios baixa",15.60,11.42,33.94,13.76,102.33,7.07,4.34,1.98,13.15,0.53,8.33,19.72,2.65,24.90,35.61,6.85,83.85,1.75,36.35
"1,0 desvios baixa",15.12,10.57,32.31,12.88,98.72,6.42,4.06,1.85,11.98,0.46,8.04,18.50,2.32,23.41,33.82,6.34,79.43,1.56,34.61
"1,5 desvios baixa",14.65,9.71,30.68,12.00,95.11,5.76,3.78,1.72,10.81,0.40,7.75,17.27,1.99,21.93,32.03,5.82,75.01,1.36,32.86
"2,0 desvios baixa",14.17,8.86,29.04,11.12,91.51,5.11,3.50,1.59,9.63,0.33,7.47,16.04,1.66,20.44,30.25,5.31,70.59,1.17,31.12


## 9. Visualização dos dados nos gráficos

In [33]:
ticker = "PETR4" #Colocar a ação que quer ver o gráfico
ticker = ticker + '.SA'

ativo = yf.download(ticker, start = inicio, end = hoje)


fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(name=ticker + ' Ajustado', x=ativo.index, y=ativo['Adj Close']),row=1,col=1)
fig.add_trace(go.Candlestick(name=ticker, x=ativo.index, close=ativo['Close'], open=ativo['Open'],high=ativo['High'], low=ativo['Low']))
fig.add_hline(y=desvios.loc["0,5 desvios alta",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'+0,5 desvio -> {desvios.loc["0,5 desvios alta",ticker]}'), annotation_position="top left")
fig.add_hline(y=desvios.loc["1,0 desvios alta",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'+1,0 desvio -> {desvios.loc["1,0 desvios alta",ticker]}'), annotation_position="top left")
fig.add_hline(y=desvios.loc["1,5 desvios alta",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'+1,5 desvio -> {desvios.loc["1,5 desvios alta",ticker]}'), annotation_position="top left")
fig.add_hline(y=desvios.loc["2,0 desvios alta",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'+2,0 desvio -> {desvios.loc["2,0 desvios alta",ticker]}'), annotation_position="top left")

fig.add_hline(y=desvios.loc["0,5 desvios baixa",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'-0,5 desvio -> {desvios.loc["0,5 desvios baixa",ticker]}'), annotation_position="bottom left")
fig.add_hline(y=desvios.loc["1,0 desvios baixa",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'-1,0 desvio -> {desvios.loc["1,0 desvios baixa",ticker]}'), annotation_position="bottom left")
fig.add_hline(y=desvios.loc["1,5 desvios baixa",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'-1,5 desvio -> {desvios.loc["1,5 desvios baixa",ticker]}'), annotation_position="bottom left")
fig.add_hline(y=desvios.loc["2,0 desvios baixa",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'-2,0 desvio -> {desvios.loc["2,0 desvios baixa",ticker]}'), annotation_position="bottom left")


fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

[*********************100%***********************]  1 of 1 completed
